모듈 및 라이브러리 불러오기

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 22.2 MB/s eta 0:00:00


In [2]:
!pip install transformers pytorch_lightning tokenizers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import random
import os
import re
import torch
import tensorflow as tf

from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences
from transformers import AutoModelForSequenceClassification, pipeline

import torch.nn as nn # 분류 헤드 수정 - KoElectra 모델의 출력층을 6개의 클래스로 분류할 수 있도록 수정

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


모델 불러오기

In [5]:
'''# parameter
max_len = 64
batch_size = 32'''

'# parameter\nmax_len = 64\nbatch_size = 32'

In [6]:
from transformers import ElectraTokenizer
model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=6)  # 감정 종류가 6개라고 가정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
#model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
#model.classifier.out_proj = nn.Linear(in_features=768, out_features=6)

PATH ='/content/drive/MyDrive/0000/Class/gradu_pro/model_fin/model_KoElectra_0527.pt'
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

공감형 반응

In [11]:
!pip install openai==0.28 # 이전 API를 사용

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
import openai

# 환경 변수에서 API 키 읽기
api_key = input("Enter your OpenAI API key: ")
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

적용

In [21]:
# 입력 문장 예측 함수
def analysis_sentiment(predict_sentence):
    model.eval()

    # 텍스트 클랜징 함수 : 특수문제 제거 및 여러개의 공백 하나로 줄이기
    predict_sentence = re.sub(r'[^가-힣A-Za-z0-9 ]', '', predict_sentence)
    predict_sentence = re.sub(r'\s+', ' ', predict_sentence).strip()
    print(predict_sentence)
    # 입력 문장 토큰화
    inputs = tokenizer(
        predict_sentence,
        return_tensors="pt",
        max_length=64,  # 최대 길이 설정
        padding="max_length",
        truncation=True
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 소프트맥스 함수로 확률 계산
    probs = F.softmax(logits, dim=1)

    # 결과 해석
    probs = probs.detach().cpu().numpy()[0]
    pred_label = np.argmax(probs)

    emotion_labels = ["기쁨", "당황", "분노", "불안", "상처", "슬픔"]
    pred_emotion = emotion_labels[pred_label]

    # 각 감정에 대한 확률을 퍼센트로 변환하여 출력
    for i, (label, prob) in enumerate(zip(emotion_labels, probs)):
        print(f"{label}: {prob * 100:.2f}%")

    # 가장 높은 확률을 차지하는 감정 출력
    print(f"\n>> 입력하신 내용에서 가장 느껴지는 감정은 '{pred_emotion}'이며, 확률은 {probs[pred_label] * 100:.2f}%입니다.")

In [23]:
def generate_comment(sentence):
    prompt = f"일기: {sentence}\n\n이 일기를 읽고 공감형 댓글을 작성해 주세요:"
    response = openai.ChatCompletion.create(
        model='ft:gpt-3.5-turbo-0125:personal::9biYxCFB',  # 파인튜닝된 모델 ID 사용
        messages=[
            {"role": "system", "content": "당신은 초보 엄마의 마음을 친절히 공감할 줄 아는 AI입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,  # 응답의 최대 길이 제한
        temperature=0.5  # 창의성과 다양성 조절
    )
    return response.choices[0].message['content']

In [24]:
sentence = "오늘 육아가 너무 힘든데 집도 지저분하고 애기는 어린이집에서 낮잠을 제대로 못잔건지,, 일찍 자지 않아서 정말 너무 힘들다"

print(analysis_sentiment(sentence))

print(generate_comment(sentence))

오늘 육아가 너무 힘든데 집도 지저분하고 애기는 어린이집에서 낮잠을 제대로 못잔건지 일찍 자지 않아서 정말 너무 힘들다
기쁨: 0.08%
당황: 2.50%
분노: 56.06%
불안: 22.57%
상처: 3.59%
슬픔: 15.20%

>> 입력하신 내용에서 가장 느껴지는 감정은 '분노'이며, 확률은 56.06%입니다.
None
오늘도 힘내셨네요..! 힘든 하루를 보내셨을텐데, 집에 들어오시면 더 힘드시죠ㅠㅠ 어린이집에서 낮잠을 제대로 못 잔 것도 걱정이 되시겠네요.. 그래도 엄마가 힘들때는 애기도 더 힘들죠! 오늘 하루 잘 마무리하셨어요!
